In [1]:
from diskcache_class import db
from lda_mallet import *
from pre_processing import *
from os import walk
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [30]:
#we are going to run this on 4 different dataset 1. EDML 2. AP 3. Newsgroup 4.Covid-Tweet


############################## reading data for ED corpus
# datafolder = './data/ed_recovery_formatted/Excel'
# #datafolder = 'ed_recovery_topicmodel'
# df = pd.DataFrame(columns=['url','type','photo','date','tags','notes','text','photo_url','reblogged','blog_name'])

# for dirpath,dirnames,filenames in walk(datafolder):
#   for filename in filenames:
#     if filename.endswith('.xlsx'):
#       t = pd.read_excel(datafolder+'/'+filename,names=['url','type','photo','date','tags','notes','text','photo_url','reblogged'])
#       blog_name = t.iloc[0,0].split(':')[1]
#       t['blog_name'] = blog_name
#       df = df.append(t.iloc[3:,:],ignore_index=True)
#       print('blog:{0}   with posts:{1}    and reblogs:{2}  '.format(filename,len(t),len(t[t.reblogged=='yes'])))


# print('number of blogs: {0} - number of posts: {1}'.format(len(set(df.blog_name)),len(df)))
# print('out of {0} documents, {1} are reblogged.'.format(len(df),len(df[df.reblogged == 'yes'])))

# #finding reblogged texts
# texts = sorted(df.text) #sort them to keep smallest post (perhaps original one) at first
# re_texts = []

# while len(texts):
#   t = [texts.pop(0)]#pop first text and find it!
#   if t[0] == ' ' or len(t[0].split())<3: 
#     continue #almost nothing to look
#   i = 0
#   while i<len(texts):
#     if t[0] in texts[i]:
#       t.append(texts.pop(i))
#     else:
#       i += 1
#   if len(t) > 1:
#     re_texts.append(t)


# print('number of unique reblogged texts: {0}'.format(len(re_texts)))
# print('number of unique string in all texts: {0}'.format(len(set(df.text))))

# extra_stopwords = ['isnt','want','cant','wanna','im','could','ive','would','dont','get','also','us','thats','got','ur','wanted',
#                    'may', 'the', 'just', 'can', 'think', 'damn', 'still', 'guys', 'literally', 'hopefully', 'much', 'even', 'rly', 'guess', 'anon']#anything with a length of one
                   

# '''pre-processing'''
# # original_doc_set = list(df.text[df.photo=='no'])
# sel_df = df[df.photo=='no'] #extracting only-text posts
# doc_list = list(sel_df.text)


# ##############reading AP corpus
# text_df = ap_corpus('./data/ap.txt')
# doc_list = list(text_df.text_cleaned)

##############reading Newsgroup corpus
text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
doc_list = list(text_df.text_cleaned)


#loading ref corpus for coherene score for lda_mallet
wiki_docs = loading_wiki_docs('./data/wiki_sampled_5p.txt')
#doing pre-processing on wiki-pedia documents
pre_processed_wiki, _ = preprocess_data(wiki_docs)
wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
del wiki_docs

In [31]:
#removing terms that are not in Wikipedia ref-corpus
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords={})
#generate vocabulary and texts
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

#finding stopwords that are not in Wikipedia and removing those
extra_stopwords = set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys()))
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords=extra_stopwords)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

In [ ]:
#running many many LDA and storing their pair of terms

start = 15; limit=26; step = 1
runs = 3
term_pairs = set()

for num_topics in tqdm(range(start, limit+1, step)):
    model_t = []
    purity_t = []
    coherence_t = []
    contrast_t = []
    for r in range(runs):
        #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=vocab_dict,optimize_interval = 25,workers=1)

        #storing top_terms
        for tn in range(num_topics): 
            tt = model.show_topic(tn,topn=20)

            #saving top_terms and their counts
            top_terms = [i[0] for i in tt]

            #making pair terms
            for i in range(len(top_terms)):
                for j in range(i+1,len(top_terms)):
                    term_pairs.add((top_terms[i],top_terms[j]))
    

  0%|                                                    | 0/12 [00:00<?, ?it/s]Mallet LDA: 15 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 12555
total tokens: 2137668
<10> LL/token: -9.52915
<20> LL/token: -9.0343
<30> LL/token: -8.91312
<40> LL/token: -8.85853

0	3.33333	god people jesus christian life bible religion church man christians christ faith word love truth paul true religious things hell 
1	3.33333	mr people time make president questions made support group work question groups back money service part general give answer asked 
2	3.33333	windows drive dos system card window problem scsi run disk pc mb mac memory set running program machine file apple 
3	3.33333	ca uk mail article ac cs apr michael net david post cc email steve list phone au internet university send 
4	3.33333	ax max gv bf pl tm di ei um wm cx ah mu mv hz ww mc mt mq ey 
5	3.33333	file image software graphics ftp version files information color video mail code data bit format computer image

<210> LL/token: -8.7705
<220> LL/token: -8.76927
[beta: 0.02276] 
<230> LL/token: -8.73044
<240> LL/token: -8.73797

0	2.67888	god people jesus christian bible life church religion christians christ man faith word love true paul truth things christianity words 
1	2.99429	people make mr time president made money states work number year group support long groups back free questions general part 
2	2.77484	windows drive dos system card problem pc mac scsi disk mb apple memory ibm running bit software work hard video 
3	3.20831	article ca apr cs uk mail ac university michael cc net au david john steve org post internet email wrote 
4	0.70411	ax max gv bf pl di tm ei um cx wm mu mv ah ey ww mw lk mk mq 
5	2.64799	file image program files graphics ftp software version window information color list mail code server data display format sun send 
6	3.04998	time power back high good problem work thing make find lot long things water light line sound put low ground 
7	2.44855	key government chip 

[beta: 0.04481] 
<400> LL/token: -8.55216
<410> LL/token: -8.53598
<420> LL/token: -8.5375
[beta: 0.04711] 
<430> LL/token: -8.51402
<440> LL/token: -8.51312

0	0.30988	god people jesus christian bible church christ christians life religion faith true man word point time question things love good 
1	0.56752	people president mr make money year time states public made government state american national free number support years care work 
2	0.31418	windows drive dos card system scsi pc mb mac disk software apple video memory hard monitor mhz os bit work 
3	0.95648	article ca apr cs mail university uk ac michael david cc email org mark internet net post john au news 
4	0.003	ax max gv bf pl di tm ei cx um wm ey mu ww ah mv mw lk wt mk 
5	0.29686	file image program files window ftp graphics software version server data code display windows color images format jpeg system information 
6	0.81509	time good back problem make thing things power work article lot long point find put people bad hi

[beta: 0.05823] 
<600> LL/token: -8.44311
<610> LL/token: -8.439
<620> LL/token: -8.43767
[beta: 0.05919] 
<630> LL/token: -8.43517
<640> LL/token: -8.43455

0	0.10339	god people jesus christian bible article church christ christians religion life faith true time point good question word things man 
1	0.11156	president people government mr states money year national public american united state make clinton time made general private years support 
2	0.11479	windows drive dos card system scsi mb pc mac disk apple software memory video problem hard bit monitor mhz os 
3	0.30108	article ca mail university apr cs uk ac internet email au michael phone list org david send cc fax mark 
4	0.00164	ax max gv bf pl di tm ei cx um wm ey mu ww ah mv mw lk wt mk 
5	0.09179	file image program files window ftp graphics software windows version server data code display jpeg images format color bit set 
6	0.30749	time good back people make problem things thing long article work lot point power find year

[beta: 0.06297] 
<800> LL/token: -8.41896
<810> LL/token: -8.41914
<820> LL/token: -8.41765
[beta: 0.06332] 
<830> LL/token: -8.41823
<840> LL/token: -8.41661

0	0.0802	god people jesus christian bible article church christ christians religion life faith time true good point things question word man 
1	0.07215	people president government mr states year national money american public state united make time clinton made years general private tax 
2	0.09235	windows drive dos card system scsi mb pc mac disk software apple problem video memory hard mhz monitor bit work 
3	0.19982	article mail ca university uk ac cs apr email internet phone send au fax org list computer mark michael david 
4	0.00144	ax max gv bf pl di tm ei cx um wm ey mu ww ah mv mw lk wt mq 
5	0.06886	file image program window files ftp graphics version software server windows data display code jpeg color images format bit set 
6	0.21426	time good back make people problem thing things work long power lot find article point

...

<210> LL/token: -8.72475
<220> LL/token: -8.72419
[beta: 0.01995] 
<230> LL/token: -8.67351
<240> LL/token: -8.68164

0	2.52692	good cc sale hp price buy mail interested book jim offer scott great sell copy man ohio st original black 
1	2.27275	key government public chip law clipper encryption system security private keys phone clinton bill insurance technology house privacy money algorithm 
2	2.30449	health medical study research food disease drugs effect problems rate problem results high drug major risk test treatment cancer number 
3	1.50886	uk ac au de ah cx mc ww mm hz mv tu pl mr ms sp md mt nz mi 
4	2.6217	people gun point evidence reason article question make true person agree wrong guns argument fact claim case moral laws opinion 
5	2.66517	power uiuc problem article make line question work cso back radio find time sound current thing light case high real 
6	2.32	space car nasa gov bike earth cars engine henry launch moon dod cost air orbit miles speed shuttle toronto mission

[beta: 0.03686] 
<400> LL/token: -8.52721
<410> LL/token: -8.51415
<420> LL/token: -8.51713
[beta: 0.03887] 
<430> LL/token: -8.50127
<440> LL/token: -8.5044

0	0.30954	sale price car buy mail good offer sell hp insurance interested shipping st cc condition dealer cover cd sold pay 
1	0.19004	government key clipper encryption chip law president public security keys mr clinton system private house privacy bill federal escrow enforcement 
2	0.11222	health medical disease drugs drug cancer msg patients food doctor treatment pitt aids article medicine study gordon hiv diet studies 
3	0.04024	db cx ww ah lk pl mk mv hz mw uw di mu bh mc ck md um mp mt 
4	0.4636	people article gun evidence point question true make reason claim argument guns fact person wrong case moral control atheists religion 
5	0.31527	power radio water work ground oil battery high phone light line circuit signal wire current problem supply low heat cable 
6	0.21355	space nasa car gov earth cars engine henry launch orbit 

[beta: 0.0495] 
<600> LL/token: -8.44253
<610> LL/token: -8.43909
<620> LL/token: -8.43826
[beta: 0.05019] 
<630> LL/token: -8.43484
<640> LL/token: -8.43318

0	0.0801	car sale cars price engine good hp article sell buy insurance miles condition ford speed offer st shipping dealer front 
1	0.06333	key government clipper encryption president chip public law security keys mr clinton system private privacy house escrow des enforcement algorithm 
2	0.03595	health medical disease article msg cancer drug patients drugs food treatment doctor pitt aids medicine study gordon hiv diet effects 
3	0.01194	db cx ww ah pl mv mk lk di hz mw uw mu ck bh mc ei md mt mp 
4	0.1133	people gun article evidence guns law point moral make question morality person argument control atheists reason atheism true religion state 
5	0.07764	power radio water work battery ground high phone circuit article wire light oil ca supply signal heat current problem audio 
6	0.0483	space nasa gov earth henry launch orbit moon

[beta: 0.0536] 
<800> LL/token: -8.4218
<810> LL/token: -8.41984
<820> LL/token: -8.419
[beta: 0.05374] 
<830> LL/token: -8.41974
<840> LL/token: -8.41745

0	0.05885	car sale cars engine price article good hp miles speed oil buy insurance ford st dealer sell drive condition offer 
1	0.04799	key government clipper encryption president chip public law security keys mr system clinton private privacy house escrow des article people 
2	0.03016	health medical article disease drug cancer msg patients drugs food doctor treatment pitt aids medicine study insurance hiv gordon diet 
3	0.00889	db cx ww lk ah pl mk mv hz mw uw di ck bh md mt mc mp um ei 
4	0.07298	gun people article guns law evidence moral make morality point control state true atheism argument atheists question crime bill objective 
5	0.05674	power radio article water battery phone ground ca work high circuit wire current audio signal cable heat light low radar 
6	0.03944	space nasa gov earth henry launch orbit moon shuttle articl

[beta: 0.05471] 
<1000> LL/token: -8.41389

Total time: 3 minutes 58 seconds
Mallet LDA: 17 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 12555
total tokens: 2137668
<10> LL/token: -9.56471


In [40]:
#converting set to list
term_pairs_ls = [[i[0],i[1]] for i in list(term_pairs)]

In [41]:
#computing Coherence for all term pairs
cscore = CoherenceModel(topics=term_pairs,dictionary=wiki_vocab_dict,texts=pre_processed_wiki,coherence='c_npmi',processes=3,topn=2).get_coherence_per_topic()


In [44]:
#Loading the DB
npmi_db = db('./db/wiki_5p_old')


Load NPMI coherence DB. 
Number of keys : 139454


In [43]:
#writing into the DB
for i in tqdm(range(len(term_pairs_ls))):
    #if the key does not exist, insert it
    try:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])]
    except:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])] = cscore[i]
    #do the other combination
    try:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])]
    except:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])] = cscore[i]

100%|█████████████████████████████████████| 5727/5727 [00:01<00:00, 3004.69it/s]


In [28]:
# for k in tqdm(npmi_db.db.iterkeys()):
#     try:
#         npmi_db.db[(k[1],k[0])]
#     except:
#         npmi_db.db[(k[1],k[0])] =  npmi_db.db[k]

125087it [00:23, 5268.58it/s]
